In [1]:
# important modules for this HW
import bs4 # doc: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import requests as rq 
import re
import time
# previous useful modules
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [2]:
import copy
import datetime

# Loading the 2 dataset

In [ ]:
df1=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_1.csv')

In [ ]:
df2=pd.read_csv('/home/stefano/Dropbox/Universita/Losanna/Exam/Anno2/DataAnalysis/main_database_2.csv')

In [ ]:
df=pd.concat([df1,df2],axis=0)

In [ ]:
df.iloc[0].link

In [ ]:
df.head()

In [ ]:
df[df.nom=='Baumer Peter']

In [ ]:
a=list(df.cat.value_counts().index)

In [ ]:
a

In [ ]:
st=""
for i,item in enumerate(a):
    if i>1000 and i<1500:
        st+=item
        st+=" "
st

In [ ]:
df['index'] = np.arange(len(df))
df = df.set_index('index')

# Links2run integration

### Loading links2run

In [ ]:
links2runs=pd.read_csv('../datasets/links2runs.csv')
del links2runs['Unnamed: 0']

In [ ]:
links2runs=links2runs.set_index(links2runs.URL)

In [ ]:
links2runs.head(3)

In [ ]:
links2runs.loc['http://services.datasport.com/2000/lauf/tdcn']

In [ ]:
links2runs.index.value_counts()

### Process link

In [ ]:
df.link=df.link.apply(lambda x: '/'.join(x.split('/')[:-1]))

In [ ]:
df.head(2)

### Groupby link

In [ ]:
links=list(df.link.value_counts().index)

In [ ]:
def add_date_name_to_group(x):
    link=x.link.iloc[0]
#     print(link)
#     print('\n\n\n\n')
    row_links2runs=links2runs.loc[link]
    try:
        row_links2runs.shape[1]
        if row_links2runs.shape[0]>1:
            row_links2runs=row_links2runs.iloc[0]
    except:
        pass
#     print(row_links2runs.shape)
    try:
        x['Date']=row_links2runs.Date
    except:
        print(row_links2runs)
        print(x)
        raise()
    x['Name']=row_links2runs.Name
    return x

In [ ]:
df.head(2)

In [ ]:
grouped_link=df.groupby(df.link)

In [ ]:
df=grouped_link.apply(add_date_name_to_group)

In [ ]:
del df['link']

In [ ]:
df.head(2)

# Delete duplicates
Some links of games are counted twice. In this case we check for duplicated rows and we remove them

In [ ]:
df=df.drop_duplicates(keep='first')

# Process temps-retards

In [ ]:
class Time:
    def __init__(self,time,mseconds=True):
        self.days,self.hours,self.minutes,self.seconds,self.mseconds=self.parse_time(time,split=True,mseconds=mseconds)
        
    def parse_time(self,time,check_only=False,split=True,mseconds=True):
        ''' Return a parsing of the time
        '''
        try :
            time.hours
            return (time.days,time.hours,time.minutes,time.seconds,time.mseconds)
        except:
            pass
        if split:
            time=time.split(' ')[0]
        if time.count(',')==0 and not check_only and mseconds:
            raise()
        time=re.split("[:.,]+",time)
        if not mseconds:
            time=time+[0]
        while len(time)<4:
            time=[0]+time
        hours,minutes,seconds,mseconds=[float(x) for x in time]
        hours+=int(minutes/60)
        minutes=minutes%60
        days=int(hours/24)
        hours=hours%24
        if not check_only:
            return (days,hours,minutes,seconds,mseconds)
        
    def get_seconds(self):  
        return self.days*3600*24+self.hours*3600 + self.minutes*60 + self.seconds + self.mseconds/1000
    
    def __str__(self):
        return str(int(self.days))+';'+str(int(self.hours))+':'+str(int(self.minutes))+'.'+str(int(self.seconds))+','+str(int(self.mseconds))
    
    def __div__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()-seconds_other
    
    def __truediv__(self,other):
        try:
            seconds_other=other.get_seconds()
        except:
            return False
        return self.get_seconds()/seconds_other
    

In [ ]:
dict_problems={}

In [ ]:
def apply_time(x,mseconds=True):
    try:
        time=Time(x,mseconds=mseconds)
    except:
        try:
            dict_problems[x]+=1
        except:
            dict_problems[x]=1
        return False
    return time

In [ ]:
df.temps=df.temps.apply(apply_time)

In [ ]:
df.retard=df.retard.apply(apply_time)

In [ ]:
df.pace=df.pace.apply(apply_time,args=(False,))

In [ ]:
print(Time('95.12,2'))

In [ ]:
a=Time('48.12,2')
b=Time('4.30,0')
print(a,b)

In [ ]:
b.get_seconds()

In [ ]:
a.get_seconds()

In [ ]:
a/b

In [ ]:
df['Distance']=df.temps/df.pace

In [ ]:
df.head(2)

In [ ]:
df_old=df.copy()

In [ ]:
df=df[df.Distance>0]

In [ ]:
df_old.shape

In [ ]:
df=df_old[df_old.Distance>0]

In [ ]:
df.head(3)

# Parse Date

In [ ]:
df['Year']=df.Date.apply(lambda x: x.split('.')[3])

In [ ]:
df['Month']=df.Date.apply(lambda x: x.split('.')[2])

In [ ]:
df.head(2)

In [ ]:
df[(df.Name=='Jungfrau-Marathon, Interlaken') & (df.an=='1958') & (df.Year=='2012')]

# Sex Male-Female

### Get names database

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
gender_names=pd.read_csv('gender_names.csv')

In [ ]:
del gender_names['Unnamed: 0']

In [ ]:
gender_names=gender_names.set_index(gender_names.name)

In [ ]:
del gender_names['name']

In [ ]:
gender_names.head(2)

### Category from sex database

In [ ]:
category_list=list(df.cat.value_counts().index)

In [ ]:
len(category_list)

In [ ]:
category_dict_sex={}

In [ ]:
cat_to_check=['B','D','F40','F/3','H','12','10-F20']

In [ ]:
for i,cat in enumerate(category_list):
    if cat in category_dict_sex:
        continue
    names=list(df[df.cat==cat].nom.values)
    not_found=0
    male=0
    female=0
    for index,name in enumerate(names):
        splitted=re.split("[- ]+",name)
        found=0
        for name in splitted:
            try:
                row=gender_names.loc[name.lower()]
                male+=row.male_prob
                female+=row.female_prob
                found+=1
            except:
                pass
    if male!=0 or female!=0:
        male_perc=male/(male+female)
        category_dict_sex[cat]=male_perc
        if male_perc<0.85 and male_perc>0.15:
            if cat not in cat_to_check:
                cat_to_check.append(cat)
#             print('Category',cat,'Perc:',male_perc)
    if i%100==0:
#         print()
        print(i)
#         print()
    if i>7000:
        break

In [ ]:
len(cat_to_check)

In [ ]:
len(category_dict_sex)

In [ ]:
cat_parsed=0
for cat in category_dict_sex:
    if category_dict_sex[cat]>0.85:
        cat_parsed+=1
        category_dict_sex[cat]='M'
    elif category_dict_sex[cat]<0.15:
        cat_parsed+=1
        category_dict_sex[cat]='F'

In [ ]:
category_dict_sex_copy=copy.deepcopy(category_dict_sex)

In [ ]:
import json

with open('category_dict_sex.json', 'w') as f:
    json.dump(category_dict_sex, f)

In [ ]:
import json

with open('category_dict_sex.json') as f:
    category_dict_sex = json.load(f)

### Improvements

In [ ]:
category_dict_sex=copy.deepcopy(category_dict_sex_copy)

In [ ]:
to_exclude=['P-W','D','SM2/D','W','P/W']

In [ ]:
cat_parsed=0
for cat in category_dict_sex:
    if cat in to_exclude:
        continue
    if category_dict_sex[cat]=='M' or category_dict_sex[cat]=='F':
        cat_parsed+=1
        continue
    cat_splitted=re.split("[-/]+",cat)
    for i in cat_splitted:
        if len(i)==3 and (i[0]=='M' or i[0]=='H') and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        elif len(i)==3 and (i[0]=='F' or i[0]=='W')  and i[1:].isdigit():
#             print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Fille' in i or 'Women' in i or 'Dam' in i:
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
        if 'Gar' in i or 'Men' in i or 'Hom' in i: 
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='M'
            cat_parsed+=1
            break
        if i=='W' or i=='D' or i=='H-U20W' or i=='H-WJ':
            print(cat,category_dict_sex[cat])
            category_dict_sex[cat]='F'
            cat_parsed+=1
            break
cat_parsed

In [ ]:
df[df.cat=='Sch-D-M']

### Final classification - accurate

In [ ]:
del df['sex']

In [ ]:
def add_sex(x):
    cat=x.cat.iloc[0]
    try:
        sex=category_dict_sex[cat]
        if sex=='M' or sex=='F':
            x['sex']=sex
        else:
            x['sex']=False
    except:
        x['sex']=False
    return x

In [ ]:
grouped=df.groupby(df.cat)

In [ ]:
df=grouped.apply(add_sex)

In [ ]:
df.sex.value_counts()

In [ ]:
df_sex=df[df.sex!=False]

In [ ]:
df=df_sex.copy()

In [ ]:
df_sex.head(2)

In [ ]:
df.shape

In [ ]:
df_sex.shape

In [ ]:
min(df.Year)

In [ ]:
min(df_sex.Year)

In [ ]:
df=df.rename(columns={'Year':'RaceYear','Month':'RaceMonth'})

In [ ]:
df.head(2)

In [ ]:
df.sex.value_counts()

# Parse year (roughly eliminating clearly bad entries)

In [ ]:
df.an=df.an.apply(lambda x: False if (x=='??' or x=='????' or x=='xxxx')  else x)

In [ ]:
df.an=df.an.apply(lambda x: int(x))

In [ ]:
df=df[df.an<2017]

In [ ]:
df=df[(df.an>1870) | (df.an<100)]

In [ ]:
df[(df.Name=='Jungfrau-Marathon, Interlaken') & (df.an==1958) & (df.RaceYear=='2012')]

# Columns names

In [ ]:
df.head(1)

In [ ]:
df.columns.values

In [ ]:
df=df[['Name','Date','RaceYear','RaceMonth','cat','Distance','nom','sex','an','lieu','rang','temps','retard','pace']]

In [ ]:
df.head()

In [ ]:
df.columns=['Race','Date','RaceYear','RaceMonth','Category','Distance','Name','Sex','Year','LivingPlace','Rank','Time','Delay','Pace']

In [ ]:
df=df.reset_index()

In [ ]:
del df['index']

In [ ]:
df.head(2)

In [ ]:
df.shape

# Distance

In [ ]:
df.Distance=df.Distance.apply(lambda x: float(x))

In [ ]:
df['round']=df.Distance.round()

In [ ]:
df.Distance=df.Distance.apply(lambda x: x if (abs(x-round(x)))>0.065 else round(x))

In [ ]:
df.Distance=df.Distance.apply(lambda x: 21.0975 if (abs(x-21.0975))<0.1 else x)

In [ ]:
df.Distance=df.Distance.apply(lambda x: 42.195 if (abs(x-42.195))<0.1 else x)

In [ ]:
df.Distance.value_counts()

In [ ]:
del df['round']

In [ ]:
df.head(5)

In [ ]:
df.shape

# Adding weather informations

In [ ]:
weather_db=pd.read_csv('../datasets/races-information-weather.csv')
del weather_db['Unnamed: 0']
del weather_db['Unnamed: 0.1']

In [ ]:
weather_db.head(2)

In [ ]:
weather_db['RaceYear']=weather_db.Date.apply(lambda x: x.split('.')[3])

In [ ]:
gr=df.groupby([df.RaceYear,df.Race])

In [ ]:
def merge_weather_info(x):
    race=x.iloc[0].Race
    date=x.iloc[0].RaceYear
    weather=weather_db[(weather_db.Name==race) & (weather_db.RaceYear==date)].iloc[0]
    x['Place']=weather.Place
    x['MinTemp']=weather.min_temp
    x['MaxTemp']=weather.max_temp
    x['Weather']=weather.weather_desc
    x['RaceID']=weather.URL
    
    return x

In [ ]:
df=gr.apply(merge_weather_info)

In [ ]:
df.shape

In [ ]:
df.head(2)

In [ ]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear=='2012') & (df.Year==1958)]

# Parse times and select only meaningful features

In [ ]:
full_df=df.copy()

In [ ]:
base=datetime.datetime.strptime('0:0.0,0','%H:%M.%S,%f')

In [ ]:
full_df.Time=full_df.Time.apply(lambda x:str(x))

In [ ]:
full_df.Pace=full_df.Pace.apply(lambda x:str(x))

In [ ]:
def modify_to_parse_day(x):
    a=str(int(x[0])+1)
    return a+x[1:]

In [ ]:
full_df.Pace=full_df.Pace.apply(modify_to_parse_day)

In [ ]:
full_df.Time=full_df.Time.apply(modify_to_parse_day)

In [ ]:
full_df['Pace'] = (pd.to_datetime(full_df.Pace, format='%d;%H:%M.%S,%f')-base)

In [ ]:
max(full_df.Time)

In [ ]:
max(full_df.Distance)

In [ ]:
full_df['Time'] = (pd.to_datetime(full_df.Time, format='%d;%H:%M.%S,%f')-base)

In [ ]:
del full_df['Delay']

In [ ]:
full_df.head()

# Better distance processing

In [ ]:
df.Race.value_counts()

In [ ]:
full_df2=full_df.copy()

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: x if x<4 or (abs(1-x/round(x)))>0.0055 else round(x))

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: 21.0975 if x>4 and (abs(1-x/21.0975))<0.0055 else x)

In [ ]:
full_df2.Distance=full_df2.Distance.apply(lambda x: 42.195 if x>4 and (abs(1-x/42.195))<0.0055 else x)

In [ ]:
full_df2.groupby(full_df2.RaceYear).count()

In [ ]:
df=full_df2.copy()

In [ ]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear=='2012') & (df.Year==1958)]

# Year - trying to get better data

In [ ]:
years_values=df.Year.value_counts().index.values
years_values[years_values<120]

In [ ]:
df.Year=df.Year.apply(lambda x: 0 if x==2 else x)

In [ ]:
df[df.Year>1900].shape

In [ ]:
df[df.Year<1900].shape

In [ ]:
reduced_df=df[df.Year<120]

In [ ]:
reduced_df[reduced_df.Year==25]

In [ ]:
reduced_df.groupby(reduced_df.Year).count()

If in the name a comma is present, it is possible that the year is misunderstood.

In [ ]:
names=reduced_df.Name
list_names=[]
for name in names:
    if ',' in name:
        list_names.append(name)

In [ ]:
df['CommaInName']=df.Name.str.contains(',')

In [ ]:
df['Less120']=df.Year<120

In [ ]:
df['FalseYear']=df.CommaInName & df.Less120

In [ ]:
def false_year(x):
    if x.iloc[0].FalseYear:
        x['Year']=float('nan')
    return x

In [ ]:
df=df.groupby(df.FalseYear).apply(false_year)

In [ ]:
del df['CommaInName']
del df['Less120']
del df['FalseYear']

In [ ]:
df.head(4)

In [ ]:
reduced_df=df[df.Year<120]

In [ ]:
list(reduced_df.Year.value_counts().index)

In [ ]:
df.Year=df.Year.apply(lambda x: float('nan') if x<10 else x)

In [ ]:
min(df.Year)

In [ ]:
df.Year=df.Year.apply(lambda x: x+1900 if x<120 else x)

In [ ]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear=='2012') & (df.Year==1958)]

# Output
First output

In [ ]:
df_output=df.copy()

In [ ]:
df_output.shape

In [ ]:
df_output.head()

In [ ]:
df_output['Name']=df_output.Name.apply(lambda x: x.split(',')[0])

In [ ]:
df_output['len_name']=df_output.Name.apply(lambda x: len(x.rstrip().lstrip().split(' ')))

In [ ]:
df_output['Name']=df_output.Name.apply(lambda x: x.split('+')[0])

In [ ]:
df_output

In [ ]:
df_output.len_name.value_counts()

In [ ]:
df_output[df_output.len_name==6]

In [ ]:
df_output.groupby(df_output.RaceYear).count()

In [ ]:
df.groupby(df.RaceYear).count()

In [ ]:
df_output.to_pickle('../datasets/intermediate_dataset.pickle')

# Solve problem 2 runners with same name in the same race

Strategy - study duplicates:  
- Remove all fully duplicated entries (same player in different categories)
- Keep entries with different living place (different players)
- To decide what to do if same place and different times

In [ ]:
df_output.Race.value_counts()

In [ ]:
df_output=pd.read_pickle('../datasets/intermediate_dataset.pickle')

In [ ]:
df=pd.read_pickle('../datasets/intermediate_dataset.pickle')

In [ ]:
df_output[(df_output.Race=='Jungfrau-Marathon, Interlaken') & (df_output.Year==1958) & (df_output.LivingPlace=='Ostermundigen')]

In [ ]:
df_red=df_output.copy()#[df_output.Race=='Course de l\'Escalade, Genève']

In [ ]:
df_red.shape

In [ ]:
grouped=df_red.groupby([df_red.RaceID,df_red.Name,df_output.Year])

In [ ]:
duplicates=grouped.filter(lambda x: len(x)>1)

In [ ]:
duplicates.shape

In [ ]:
duplicates

In [ ]:
duplicates.Race.value_counts()

In [ ]:
a=df_output[(df_output.Race=='Jungfrau-Marathon, Interlaken') & (df_output.RaceYear==2012)]

In [ ]:
a[a.Name.to_series().str.contains('Baumer').tolist()]

In [ ]:
a[a.Name=='Peter Baumer']

In [ ]:
df_output[df_output.Name=='Baumer Peter']

In [ ]:
df_output.Race.value_counts()

In [ ]:
df_output[(df_output.Name=='Baumer Peter')]

In [ ]:
duplicates.columns

In [ ]:
d2=df_output[df_output.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'],keep=False)]

In [ ]:
d2=df_output[df_output.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace'],keep=False)]

In [ ]:
d2.shape

In [ ]:
d2[d2.Race=='Zürcher Silvesterlauf, Zürich']

In [ ]:
d2.Distance.value_counts()

In [ ]:
d1=duplicates[duplicates.duplicated(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'],keep=False)]

In [ ]:
d1.shape

In [ ]:
d1.Race.value_counts()

In [ ]:
4961-4580

In [ ]:
duplicates.shape

In [ ]:
dup=duplicates.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'])

In [ ]:
dup.shape

In [ ]:
dup=duplicates.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth','Name', 'Sex',
       'Year', 'LivingPlace'],keep=False)

In [ ]:
dup.shape

In [ ]:
dup[dup.Race=='Jungfrau-Marathon, Interlaken']

In [ ]:
df_output=df_output.drop_duplicates(subset=['Race', 'Date', 'RaceYear', 'RaceMonth', 'Distance', 'Name', 'Sex',
       'Year', 'LivingPlace', 'Time', 'Pace'])

In [ ]:
df_output[(df_output.Race=='Jungfrau-Marathon, Interlaken') & (df_output.RaceYear=='2012') & (df_output.Year==1958)]

In [ ]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear=='2012') & (df.Year==1958)]

# Sex verification
The sex classification seems working

In [ ]:
lau=df_output[(df_output.RaceID=='http://services.datasport.com/2007/lauf/Greifenseelauf')&(df_output.Distance>6)]

In [ ]:
lau.Distance.value_counts()

In [ ]:
lau.Sex.value_counts()

In [ ]:
df_output.RaceID.value_counts()

In [ ]:
df_output.shape

# Fixing not good categories

In [ ]:
df_output.shape

In [ ]:
df=df_output.copy()

In [ ]:
df.shape

In [ ]:
cat_list=list(df.Category.value_counts().index)

In [ ]:
df[df.Category=='MTBM1'].head(2)

In [ ]:
cat_to_eliminate=[]
for i in cat_list:
    if 'MTB' in i:
        print(i)
        cat_to_eliminate.append(i)

In [ ]:
for i in cat_to_eliminate:
    df=df[df.Category!=i]

In [ ]:
df.shape

In [ ]:
df_output=df.copy()

In [ ]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear==2012) & (df.Year==1958)]

# Capitalization

In [ ]:
'CIAO COME STAI'.title()

In [ ]:
df=df_output.copy()

In [ ]:
df.dtypes

In [ ]:
df.RaceYear=df.RaceYear.apply(lambda x: int(x))

In [ ]:
df.RaceMonth=df.RaceMonth.apply(lambda x: int(x))

In [ ]:
df.Name=df.Name.apply(lambda x: x.title())

In [ ]:
df['index'] = np.arange(len(df))
df = df.set_index('index')

In [ ]:
df=df.reset_index()
del df['index']

In [ ]:
df[(df.Race=='Jungfrau-Marathon, Interlaken') & (df.RaceYear==2012) & (df.Year==1958)]

In [ ]:
df_output=df.copy()

# Fix names

In [ ]:
df_output.Name=df_output.Name.apply(lambda x: x.rstrip())

In [ ]:
df_output.Name=df_output.Name.apply(lambda x: x.lstrip())

In [ ]:
df_output[df_output.Name=='Baumer Peter']

# Process LivingPlace

In [3]:
df_output=pd.read_pickle('../datasets/df_added_category.pickle')

In [4]:
df_output.LivingPlace=df_output.LivingPlace.apply(lambda x: x.lstrip())
df_output.LivingPlace=df_output.LivingPlace.apply(lambda x: x.rstrip())

In [5]:
df_output=df_output[df_output.Race!='Töss-Stafette, Winterthur']

In [6]:
df=df_output.copy()

In [7]:
cantons=['AG','AI','AR','BE','BL','BS','FR','GE','GL','GR','JU','LU','NE','NW','OW','SG',
         'SH','SO','SZ','TG','TI','UR','VD','VS','ZG','ZH','GB','TV','AT','KLH','DK','DE',
         'JP','NL','VA','BE','RC','MI','BG','UBS','NO','RN','BZ','LC','FI','CO','UD','TO','MB',
        'RE','VB','US','SV','AL','PO','BO','NA','BI','SE','PT','LO','AO','FL','DC','LR','LA','SR',
         'ZS']

In [8]:
living_places=list(df.LivingPlace.value_counts().index)
proc1=living_places[:]

In [9]:
for i,word in enumerate(living_places):
    if word.isupper():
        proc1[i]=word.title()

In [10]:
processing_dict={i:j for i,j in zip(living_places,proc1)}

In [11]:
for cat1,cat2 in zip(living_places,proc1):
    for st in cantons:
        if st in cat2:
            processing_dict[cat1]=cat2.replace(st,'').lstrip().rstrip()
    processing_dict[cat1]=processing_dict[cat1].replace('-',' ').lstrip().rstrip()
    processing_dict[cat1]=processing_dict[cat1].replace('(','')
    processing_dict[cat1]=processing_dict[cat1].replace(')','')

In [12]:
b=df.copy()

In [13]:
b.LivingPlace=b.LivingPlace.apply(lambda x: processing_dict[x])

In [14]:
processing_dict={i:i for i in processing_dict.values()}

In [15]:
living_places=list(b.LivingPlace.value_counts().index)

In [16]:
living_places_start=living_places[:100]
living_places=living_places[100:]

In [17]:
for i in living_places_start:
    for j in living_places:
        if i in j:
            processing_dict[j]=i

In [18]:
b.LivingPlace=b.LivingPlace.apply(lambda x: processing_dict[x])

In [19]:
processing_dict={i:i for i in processing_dict.values()}

In [20]:
for i in processing_dict.keys():
    try:
        if i[1]==' ':
            processing_dict[i]=processing_dict[i][2:].lstrip()
    except:
        pass

In [21]:
b.LivingPlace=b.LivingPlace.apply(lambda x: processing_dict[x])

In [22]:
processing_dict={i:i for i in processing_dict.values()}

In [23]:
len(list(b.LivingPlace.value_counts().index))

46857

In [24]:
living_places=list(b.LivingPlace.value_counts().index)

In [25]:
living_places_start=living_places[:5000]
living_places=living_places[5000:]

In [26]:
for i in living_places_start:
    for j in living_places:
        if i in j and len(i)>1:
            processing_dict[j]=i

In [27]:
b.LivingPlace=b.LivingPlace.apply(lambda x: processing_dict[x])

In [28]:
processing_dict={i:i for i in processing_dict.values()}

In [29]:
len(list(b.LivingPlace.value_counts().index))

36166

In [30]:
import unicodedata
for i in processing_dict.keys():
    s=processing_dict[i]
    processing_dict[i]=''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

In [31]:
b.LivingPlace=b.LivingPlace.apply(lambda x: processing_dict[x])

In [32]:
processing_dict={i:i for i in processing_dict.values()}

In [33]:
len(list(b.LivingPlace.value_counts().index))

35883

In [34]:
values=list(b.LivingPlace.value_counts().values)

In [35]:
living_places=list(b.LivingPlace.value_counts().index)

In [36]:
b.shape

(1650463, 18)

In [37]:
sum(values[:5000])

1564363

In [38]:
glob=[(i,j) for i,j in zip(living_places,values)]
glob[4000:5000]

[('Unihockey Wilderswil Inte', 30),
 ('Clarafond', 30),
 ('Farnern', 30),
 ('Hofmatt, Horw', 30),
 ('Beromuenster', 30),
 ('Hausernmoos im Emmental', 30),
 ('Beriker Jogger', 30),
 ('Evires', 30),
 ('Aschaffenburg', 30),
 ('Alfter', 30),
 ('Villars le Comte', 30),
 ('Verrieres de Joux', 30),
 ('Bidogno', 30),
 ('Marchirolo ', 30),
 ('Running Team Lauwe', 30),
 ('Mustair', 30),
 ('Heddesheim', 30),
 ('Cityrunning', 30),
 ('Nespresso running team', 30),
 ('Cornaux Ne', 30),
 ('Ecole Du Mail', 30),
 ("Vers l'Eglise", 30),
 ('Hilchenbach', 30),
 ('Inverigo ', 30),
 ('Mache', 30),
 ('Landiswil', 30),
 ('Syens', 30),
 ('Kaltenbach', 30),
 ('Ton toubib se bouge 20 km', 30),
 ('Banque Privee Edmond De R', 30),
 ('Jugi Wila', 30),
 ('Contamine Sarzin', 30),
 ('Ccmj', 30),
 ('Soyhieres', 30),
 ('Flaxlanden', 30),
 ('Le Belieu', 30),
 ('Procter Et Gamble Europe', 30),
 ('Breisach', 30),
 ('Cairate ', 30),
 ('triseeland.ch', 30),
 ('Sitterdorf', 30),
 ('Schonau', 30),
 ('Toulouse', 30),
 ('Pino La

In [39]:
df=b.copy()

In [40]:
df_output=df.copy()

# Try to identify different people with same name

- Remove race Töss-Stafette, Winterthur
- Drop the duplicates with same userID, race and distance (same player, tipycally error in Datasport). In case of Jungfrau marathon I keep only 1 result.
- Keep the longest race if same userID and race but different distance

In [41]:
df=df_output.copy()

In [42]:
df['YearStr']=df.Year.apply(lambda x: str(x))

In [43]:
df['UserID']=df.Name+' '+df.YearStr+' '+df.LivingPlace

In [44]:
df['UserIDYear']=df.Name+' '+df.YearStr

In [45]:
df['DistanceRound']=df.Distance.apply(lambda x: round(x))

In [46]:
df=df.drop_duplicates(subset=['RaceID','UserID','DistanceRound'])

In [47]:
df[df.Year.isnull()].shape

(36661, 22)

In [52]:
len(set(df.Name))

466389

In [53]:
len(set(df.UserIDYear))

531426

In [54]:
len(set(df.UserID))

651713

In [55]:
df=df[df.groupby([df.RaceID,df.UserID])['Distance'].transform(max)==df['Distance']]

In [56]:
df[df.duplicated(subset=['RaceID','Name','Year'],keep=False)]

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,...,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,YearStr,UserID,UserIDYear,DistanceRound
11835,Kerzerslauf,sam. 17.03.2001,2001,3,M40,15.000000,Hayoz Hugo,M,1960.0,Rotkreuz,...,00:05:00,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,1960.0,Hayoz Hugo 1960.0 Rotkreuz,Hayoz Hugo 1960.0,15
11836,Kerzerslauf,sam. 17.03.2001,2001,3,M40,15.000000,Hayoz Hugo,M,1960.0,Dudingen,...,00:05:46,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,1960.0,Hayoz Hugo 1960.0 Dudingen,Hayoz Hugo 1960.0,15
13217,Kerzerslauf,sam. 17.03.2001,2001,3,M20,15.000000,Röthlisberger Heinz,M,1968.0,Grosshochstetten,...,00:03:41,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,1968.0,Röthlisberger Heinz 1968.0 Grosshochstetten,Röthlisberger Heinz 1968.0,15
13218,Kerzerslauf,sam. 17.03.2001,2001,3,M20,15.000000,Röthlisberger Heinz,M,1968.0,Batterkinden,...,00:06:40,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,1968.0,Röthlisberger Heinz 1968.0 Batterkinden,Röthlisberger Heinz 1968.0,15
16107,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,17/S3,17.344485,Achermann Josef,M,NaN,Geuensee,...,00:04:59,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Achermann Josef nan Geuensee,Achermann Josef nan,17
16108,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,17/S4,17.301079,Achermann Josef,M,NaN,Slvs,...,00:04:39,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Achermann Josef nan Slvs,Achermann Josef nan,17
16291,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,17/S2,17.348022,Christen Rolf,M,NaN,Olten,...,00:03:47,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Christen Rolf nan Olten,Christen Rolf nan,17
16292,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,CK,1.000000,Christen Rolf,M,NaN,Gettnau,...,00:04:31,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Christen Rolf nan Gettnau,Christen Rolf nan,1
16803,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,17/H,17.331182,Odermatt Hugo,M,NaN,Sempach,...,00:05:08,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Odermatt Hugo nan Sempach,Odermatt Hugo nan,17
16804,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,17/S2,17.336611,Odermatt Hugo,M,NaN,Oberdorf,...,00:05:06,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Odermatt Hugo nan Oberdorf,Odermatt Hugo nan,17


In [58]:
df.Race.value_counts()

Course de l'Escalade, Genève                        152815
Schweizer Frauenlauf Bern                           118906
20km de Lausanne                                     98783
Morat-Fribourg                                       91950
Kerzerslauf                                          86246
Zürcher Silvesterlauf, Zürich                        86113
Int Greifenseelauf, Uster                            71820
Zürich Marathon, Teamrun und Cityrun                 60660
Basler Stadtlauf                                     55896
Hallwilerseelauf, Beinwil am See                     48263
Jungfrau-Marathon, Interlaken                        42470
Lausanne Marathon                                    39957
Lucerne Marathon, Luzern                             29938
ASICS Bremgarter Reusslauf                           28377
SwissCityMarathon - Lucerne, Luzern                  24455
Corrida Bulloise, Bulle                              22891
Luzerner Stadtlauf                                   226

### Clustering distances

In [232]:
from sklearn.cluster import DBSCAN

def cluster_race_distances(x):
    dbscan=DBSCAN(eps=0.2,min_samples=2)
    dict_cluster={}
    dist=np.array(x.Distance.value_counts().index)
    for i in dist:
        if i==round(i) or i==42.195 or i==21.0975:
            dict_cluster[i]=i
    dista=np.array(dist).reshape(-1,1)
    dbscan.fit(X=dista)
    categories=max(list(dbscan.labels_))
    print(x.iloc[0].Race,categories)
    for i in range(categories+1):
        r=dist[dbscan.labels_==i]
        mean_r=np.mean(r)
        for j in r:
            dict_cluster[j]=mean_r
    for i in x.Distance:
        try:
            dict_cluster[i]
        except:
            dict_cluster[i]=i
            print(i)
    x['ClusterDistance']=x.Distance.apply(lambda e: dict_cluster[e])
    return x

In [225]:
df=df.groupby(df.Race).apply(cluster_race_distances)

10km Schweizer Meisterschaften Lyss 0
10km Schweizer Meisterschaften Lyss 0
10km de Payerne 2
10km, Payerne 2
20km de Lausanne 0
A travers Carouge - Run to Run 3
ASICS Bremgarter Reusslauf 7
Aegeriseelauf, Oberägeri 1
Aletsch-Halbmarathon, Bettmeralp 2
Altstätter Städtlilauf 5
Augusta Raurica Lauf, Augst BL 1
BCN Tour #1 Corcelles 3
BCN Tour #2 La Chaux-de-Fonds 3
BCN Tour #3 Dombresson 3
BCN Tour #4 Fleurier 3
BCN Tour #5 Colombier 3
BCN Tour #6 Neuchâtel 2
BCN Tour Auvernier 2
BCN Tour Boudry 2
BCN Tour Colombier 2
BCN Tour Cornaux 1
BCN Tour Couvet 2
BCN Tour Fontaine/Cernier 1
BCN Tour La Brévine 1
BCN Tour La Chaux-de-Fonds 3
BCN Tour La Chaux-de-fonds 1
BCN Tour Le Landeron 0
BCN Tour Le Locle 2
BCN Tour Les Ponts de Martel 1
BCN Tour Les Ponts-de-Martel 2
BCN Tour Môtier 2
BCN Tour Neuchâtel 6
BCN Tour Ponts-de-Martel 2
Badener Limmat-Lauf, Baden 3
Baldeggerseelauf, Hitzkirch 4
Basel City Marathon -1
Basler Marathon-Tage -1
Basler Stadtlauf 6
Bieler Lauftage, Biel/Bienne 4
Birsl

In [229]:
df.ClusterDistance=df.ClusterDistance.apply(lambda x: 21.0975 if x>21.1 and x<21.4 else x)

In [236]:
del df['Distance']

In [237]:
df=df.rename(columns={'ClusterDistance':'Distance'})

In [238]:
df=df[['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance', 'Name',
       'Sex', 'Year', 'LivingPlace', 'Rank', 'Time', 'Pace', 'Place',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'YearStr', 'UserID',
       'UserIDYear', 'DistanceRound']]

In [239]:
df.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance', 'Name',
       'Sex', 'Year', 'LivingPlace', 'Rank', 'Time', 'Pace', 'Place',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'YearStr', 'UserID',
       'UserIDYear', 'DistanceRound'],
      dtype='object')

### Go on with UserID

No clear strategy

In [258]:
df[df.duplicated(subset=['RaceID','UserIDYear'],keep=False)]

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,...,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,YearStr,UserID,UserIDYear,DistanceRound
11835,Kerzerslauf,sam. 17.03.2001,2001,3,M40,15.000000,Hayoz Hugo,M,1960.0,Rotkreuz,...,00:05:00,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,1960.0,Hayoz Hugo 1960.0 Rotkreuz,Hayoz Hugo 1960.0,15
11836,Kerzerslauf,sam. 17.03.2001,2001,3,M40,15.000000,Hayoz Hugo,M,1960.0,Dudingen,...,00:05:46,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,1960.0,Hayoz Hugo 1960.0 Dudingen,Hayoz Hugo 1960.0,15
13217,Kerzerslauf,sam. 17.03.2001,2001,3,M20,15.000000,Röthlisberger Heinz,M,1968.0,Grosshochstetten,...,00:03:41,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,1968.0,Röthlisberger Heinz 1968.0 Grosshochstetten,Röthlisberger Heinz 1968.0,15
13218,Kerzerslauf,sam. 17.03.2001,2001,3,M20,15.000000,Röthlisberger Heinz,M,1968.0,Batterkinden,...,00:06:40,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/kerzers,1968.0,Röthlisberger Heinz 1968.0 Batterkinden,Röthlisberger Heinz 1968.0,15
16107,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,17/S3,17.328611,Achermann Josef,M,NaN,Geuensee,...,00:04:59,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Achermann Josef nan Geuensee,Achermann Josef nan,17
16108,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,17/S4,17.328611,Achermann Josef,M,NaN,Slvs,...,00:04:39,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Achermann Josef nan Slvs,Achermann Josef nan,17
16291,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,17/S2,17.328611,Christen Rolf,M,NaN,Olten,...,00:03:47,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Christen Rolf nan Olten,Christen Rolf nan,17
16292,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,CK,1.000000,Christen Rolf,M,NaN,Gettnau,...,00:04:31,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Christen Rolf nan Gettnau,Christen Rolf nan,1
16803,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,17/H,17.328611,Odermatt Hugo,M,NaN,Sempach,...,00:05:08,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Odermatt Hugo nan Sempach,Odermatt Hugo nan,17
16804,Hellebardenlauf Sempach,sam. 30.06.2001,2001,6,17/S2,17.328611,Odermatt Hugo,M,NaN,Oberdorf,...,00:05:06,Sempach Stadt,NaN,NaN,NaN,http://services.datasport.com/2001/lauf/sempach,nan,Odermatt Hugo nan Oberdorf,Odermatt Hugo nan,17


In [241]:
df.UserIDYear.value_counts()

Nivola Gianpaolo 1966.0           178
Zuber Benno 1960.0                157
Gerwer Armin 1947.0               147
Meyer Dirk 1968.0                 143
Triantafillou Christos 1968.0     142
Gilles Jean-Guy 1947.0            140
Mletzkowsky Manfred 1934.0        139
Vetter Raphael 1959.0             133
Binder Peter 1956.0               128
Abgottspon Medard 1942.0          126
Ummel Beat 1960.0                 125
Liembd Kurt 1958.0                124
Jost Hansueli 1978.0              124
Calame Louis 1952.0               119
Leserf Gottlieb 1938.0            119
Scarcella Giovanni 1963.0         118
Ehrbar Jean-Luc 1957.0            117
Buzzi Andreas 1962.0              116
Fritschi Johann 1937.0            116
Senn Silvia 1959.0                115
Von Känel Christoph 1959.0        114
Guignard Roland 1957.0            113
Bader Renate 1970.0               112
Vogel Bruno 1959.0                112
Scheiber Kaspar 1939.0            112
Eichenberger Romain 1963.0        111
Plüss Fritz 

In [260]:
df[df.Name=='Hayoz Hugo'].Year.value_counts()

1960.0    28
Name: Year, dtype: int64

In [245]:
list(df[df.UserIDYear=='Nivola Gianpaolo 1966.0'].LivingPlace)

['Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Mauritius Emmen',
 'Mauritius Emmen',
 'Mauritius Emmen',
 'Mauritius Emmen',
 'Mauritius Emmen',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Mauritius Emmen',
 'Mauritius Emmen',
 'Littau',
 'Littau',
 'Mauritius Emmen',
 'Littau',
 'Littau',
 'Littau',
 'Mauritius Emmen',
 'Littau',
 'Littau',
 'Littau',
 'Mauritius Emmen',
 'Littau',
 'Mauritius Emmen',
 'Mauritius Emmen',
 'Littau',
 'Littau',
 'Littau',
 'Mauritius Emmen',
 'Littau',
 'Mauritius Emmen',
 'Littau',
 'Mauritius Emmen',
 'Littau',
 'Mauritius Emmen',
 'Mauritius Emmen',
 'Littau',
 'Littau',
 'Littau',
 'Littau',
 'Emmen',
 'Luzern',
 'Luzern',
 'Littau',
 'Luzern',
 'Luzern',
 'Luzern',
 'Luzern',
 'Luzern',
 

In [265]:
del df['DistanceRound']

In [266]:
del df['UserID']

In [277]:
df=df.rename(columns={'UserIDYear':'UserID'})

In [278]:
df.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance', 'Name',
       'Sex', 'Year', 'LivingPlace', 'Rank', 'Time', 'Pace', 'Place',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'UserID'],
      dtype='object')

In [270]:
del df['YearStr']

# Output to file

In [279]:
df_output=df.copy()

In [ ]:
del df_output['len_name']

In [280]:
df_output.columns

Index(['Race', 'Date', 'RaceYear', 'RaceMonth', 'Category', 'Distance', 'Name',
       'Sex', 'Year', 'LivingPlace', 'Rank', 'Time', 'Pace', 'Place',
       'MinTemp', 'MaxTemp', 'Weather', 'RaceID', 'UserID'],
      dtype='object')

In [281]:
df_output

,Race,Date,RaceYear,RaceMonth,Category,Distance,Name,Sex,Year,LivingPlace,Rank,Time,Pace,Place,MinTemp,MaxTemp,Weather,RaceID,UserID
0,Kerzerslauf,sam. 18.03.2000,2000,3,M20,15.000000,Abgottspon Peter,M,1974.0,Zermatt,233,01:02:25,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,Abgottspon Peter 1974.0
1,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Abplanalp Michael,M,1964.0,Bern,32,00:55:11.700000,00:03:40,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,Abplanalp Michael 1964.0
2,Kerzerslauf,sam. 18.03.2000,2000,3,M50,15.000000,Abt Werner,M,1947.0,Spiez,155,01:12:42.900000,00:04:50,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,Abt Werner 1947.0
3,Kerzerslauf,sam. 18.03.2000,2000,3,F45,15.000000,Ackermann Antoinette,F,1953.0,Alterswil,48,01:22:36.700000,00:05:30,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,Ackermann Antoinette 1953.0
4,Kerzerslauf,sam. 18.03.2000,2000,3,F50,15.000000,Ackermann Hedy,F,1946.0,Alterswil,42,01:23:29.300000,00:05:33,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,Ackermann Hedy 1946.0
5,Kerzerslauf,sam. 18.03.2000,2000,3,M40,15.000000,Ackermann Meinrad,M,1956.0,La Joux,35,00:56:53.100000,00:03:47,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,Ackermann Meinrad 1956.0
6,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Adamina Bruno,M,1962.0,Bern,26,00:54:41.700000,00:03:38,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,Adamina Bruno 1962.0
7,Kerzerslauf,sam. 18.03.2000,2000,3,M35,15.000000,Adatte Michel,M,1964.0,Asuel,48,00:56:36.600000,00:03:46,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,Adatte Michel 1964.0
8,Kerzerslauf,sam. 18.03.2000,2000,3,M20,15.000000,Adesris Yeshaneh,M,1973.0,ETH Aethiopien,23,00:49:27.300000,00:03:17,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,Adesris Yeshaneh 1973.0
9,Kerzerslauf,sam. 18.03.2000,2000,3,M50,15.000000,Adler Hanspeter,M,1949.0,Koblenz,33,01:02:23,00:04:09,Kerzers,NaN,NaN,NaN,http://services.datasport.com/2000/lauf/kerzers,Adler Hanspeter 1949.0


In [282]:
df_output.shape

(1648676, 19)

In [283]:
df_output.dtypes

Race                    object
Date                    object
RaceYear                 int64
RaceMonth                int64
Category                object
Distance               float64
Name                    object
Sex                     object
Year                   float64
LivingPlace             object
Rank                     int64
Time           timedelta64[ns]
Pace           timedelta64[ns]
Place                   object
MinTemp                float64
MaxTemp                float64
Weather                 object
RaceID                  object
UserID                  object
dtype: object

In [284]:
df_output.shape

(1648676, 19)

In [285]:
df_output.to_pickle('../datasets/df_userID.pickle')